In [1]:
#simple bert classifier

In [2]:
# !pip install transformers
# !pip install torch
# !pip install pytorch-lightning

In [3]:
#  !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.16.1%2Bcu118-cp39-cp39-win_amd64.whl (4.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.1.1%2Bcu118-cp39-cp39-win_amd64.whl (3.9 MB)


In [4]:
import torch

# If there's a GPU available...
torch.cuda.is_available()    


True

In [16]:
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [13]:
df=pd.read_json('yt_training_set.json')

df = pd.DataFrame(df)
# print(df)

#drop missing values
columns_to_dropna = ["title", "channel_title", "view_count", "tags", "description", "label"]

# dropna method drop rows with missing values 
filtered_data = df.dropna(subset=columns_to_dropna, how='any')

# print(filtered_data)
new_df=pd.DataFrame(filtered_data)



new_dataframe= pd.DataFrame({'title': new_df["title"].values,"channel_title": new_df["channel_title"].values,"view_count":new_df["view_count"].values,"tags":new_df["tags"].values,"description":new_df["description"].values,'label': new_df["label"].values})
# print(new_dataframe)
new_df=new_dataframe


# Convert the data to a pandas DataFrame
df_concat = pd.DataFrame(columns=["text", "label"])


for entry in new_df.index:
    
    title=new_df["title"].get(entry) 
    # print(title)
    
    channel_title=new_df["channel_title"].get(entry)
    
    tags=" ".join(new_df["tags"].get(entry ,[]))
    description=new_df["description"].get(entry)
    
    label=new_df["label"].get(entry)

    
    

    concat_text = f"{title} {channel_title} {tags} {description}"
    # concat_text=re.sub("(http\S+)", "", concat_text)
    
    df_concat = df_concat.append({"text": concat_text, "label": label}, ignore_index=True)


C:\Users\zacha\AppData\Local\Temp\ipykernel_23476\854133105.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_concat = df_concat.append({"text": concat_text, "label": label}, ignore_index=True)
C:\Users\zacha\AppData\Local\Temp\ipykernel_23476\854133105.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_concat = df_concat.append({"text": concat_text, "label": label}, ignore_index=True)
C:\Users\zacha\AppData\Local\Temp\ipykernel_23476\854133105.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_concat = df_concat.append({"text": concat_text, "label": label}, ignore_index=True)
C:\Users\zacha\AppData\Local\Temp\ipykernel_23476\854133105.py:44: FutureWarning: The frame.append method is deprecated and will be r

,text,label
0,"The Second Book of Enoch (Human Voice, Read-Al...",-1
1,Obama's Best Comebacks and Rebuttal Moments - ...,-1
2,BLACK MAN in a MAGA Hat & American Exceptional...,-1
3,Reagan Warned Us About Obama Calvin King Obama...,-1
4,TRUMP'S SECRET PLAN: HOW THE GOVT SHUTDOWN IS ...,-1
...,...,...
993,Chair Pulling Pranks (Gone Wrong) - 2018 !! Ji...,-1
994,Firefighters Laugh At Inappropriate Moments (E...,-1
995,Joshua Kutuny: Aaron Cheruiyot is not aware of...,-1
996,The Wandering Trader Update is BAD... so we ch...,-1


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)
# tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)
# label_encoder = LabelEncoder()

# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.fit_transform(y_test)
# print(y_train_encoded)


In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# Tokenize the text
encodings = tokenizer(df_concat['text'].tolist(), truncation=True, padding=True, max_length=512)

In [24]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
# Create the dataset
dataset = CustomDataset(encodings, df_concat['label'].tolist())


